[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nmickevicius/MCW_BIOP_03-238_MRI/blob/main/02_fourier_transforms/fourier_transforms.ipynb)

# Fourier transforms for frequency analysis

**Why we use them.** Real-world signals (audio, vibration, EEG, finance) often contain multiple oscillatory components . The Fourier tranfsorm of view says many signals can be expressed as a weighted sum of sinusoids. Inspecting those weights reveals which frequencies are present (spectral content).

**Continuous Fourier transform (CFT)**
$$
X(f)=\int_{-\infty}^{\infty} x(t)\,e^{-i2\pi ft}\,dt,
\qquad
x(t)=\int_{-\infty}^{\infty} X(f)\,e^{i2\pi ft}\,df .
$$

**Discrete Fourier transform (DFT)** for a length-$N$ sequence $x[n]$
$$
X[k]=\sum_{n=0}^{N-1} x[n]\,e^{-i 2\pi kn/N},\qquad
x[n]=\frac{1}{N}\sum_{k=0}^{N-1} X[k]\,e^{i 2\pi kn/N}.
$$
In practice the DFT is evaluated by a Fast Fourier Transform (FFT).


**Fourier series vs. transform.** For **periodic** signals (period $2\pi$ here), the Fourier **series** writes
$$
f(x)=\frac{a_0}{2}+\sum_{k=1}^{\infty}\big(a_k\cos kx+b_k\sin kx\big)
=\frac{a_0}{2}+\sum_{k=1}^{\infty} A_k\cos\!\big(kx-\phi_k\big),
$$
with $A_k=\sqrt{a_k^2+b_k^2}$ and $\phi_k=\operatorname{atan2}(b_k,a_k)$.
In the demo below we approximate a **boxcar** (rectangular) pulse with a finite number $N$ of harmonics—great for visualizing Gibbs ringing and how the scaled, phase-shifted components build the shape.

**Rectangular ↔ polar forms (for intuition about complex sinusoids).**
If $z=a+ib$, then
$$
|z|=\sqrt{a^2+b^2},\quad \arg z=\theta=\operatorname{atan2}(b,a),\quad
z=|z|\,e^{i\theta}.
$$
Multiplying by $e^{i\phi}$ corresponds to a **phase shift/rotation** by $\phi$; this is the essence of how complex exponentials encode frequency and phase.


In [ ]:
%matplotlib inline
import numpy as np
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Checkbox
from IPython.display import clear_output, display, HTML
import matplotlib.pyplot as plt


def _boxcar_periodic(x, w, phi0):
    """
    Periodic boxcar on [-pi, pi] with half-width w (0<w<=pi) and center shift phi0.
    Value 1 inside the pulse and 0 outside, repeated every 2*pi.
    """
    # Wrap (x - phi0) to [-pi, pi]
    y = (x - phi0 + np.pi) % (2*np.pi) - np.pi
    return (np.abs(y) <= w).astype(float)

def _series_coeffs_boxcar(w, phi0, K):
    """
    Fourier series coefficients for the shifted boxcar on [-pi, pi].
    Start from the centered even boxcar (only cosines), then shift by phi0.

    For the centered pulse:
      a0 = 2w/pi
      a_k0 = (2/pi) * sin(k w) / k
      b_k0 = 0

    After shift by phi0:
      a_k = a_k0 * cos(k*phi0)
      b_k = a_k0 * sin(k*phi0)
    """
    k = np.arange(1, K+1, dtype=float)
    a0 = 2*w/np.pi
    a_k0 = (2/np.pi) * np.sin(k*w) / k
    a_k = a_k0 * np.cos(k*phi0)
    b_k = a_k0 * np.sin(k*phi0)
    return a0, a_k, b_k

def boxcar_fourier_demo(N=12, w_over_pi=0.40, phi0=0.0, show_harmonics=True):
    """
    Re-rendered on each slider move (works with %matplotlib inline).
    N: number of harmonics
    w_over_pi: half-width as fraction of pi (0<w/pi<1)
    phi0: shift of the pulse center (radians)
    """
    clear_output(wait=True)

    # Grid over one period
    x = np.linspace(-np.pi, np.pi, 2000, endpoint=False)

    # Parameters
    w = np.clip(w_over_pi, 1e-3, 0.999) * np.pi
    K = int(max(1, N))

    # Target periodic boxcar
    f = _boxcar_periodic(x, w, phi0)

    # Coefficients and partial sum
    a0, a_k, b_k = _series_coeffs_boxcar(w, phi0, K)
    k = np.arange(1, K+1, dtype=float)

    # Vectorized partial sum: a0/2 + sum_k [a_k cos(kx) + b_k sin(kx)]
    s = (a0 / 2.0) + (a_k @ np.cos(np.outer(k, x))) + (b_k @ np.sin(np.outer(k, x)))

    # Plot
    plt.figure(figsize=(7, 5))
    ax = plt.gca()
    ax.set_title("Fourier Series Approximation of a Boxcar (period $2\\pi$)")
    ax.set_xlabel("x")
    ax.set_ylabel("amplitude")
    ax.set_xlim(-np.pi, np.pi)
    ax.set_ylim(-0.3, 1.3)
    ax.grid(True, linestyle=":")

    # Boxcar edges for reference
    ax.axvline((phi0 - w + np.pi) % (2*np.pi) - np.pi, linestyle="--", linewidth=1)
    ax.axvline((phi0 + w + np.pi) % (2*np.pi) - np.pi, linestyle="--", linewidth=1)

    # Target and partial sum
    ax.plot(x, f, linewidth=2, label="boxcar")
    ax.plot(x, s, linewidth=2, label=f"partial sum (N={K})")

    # Optional: show individual harmonics in amplitude–phase form
    if show_harmonics:
        # A_k = sqrt(a_k^2 + b_k^2) ; phi_k = atan2(b_k, a_k)
        A_k = np.sqrt(a_k**2 + b_k**2)
        phi_k = np.arctan2(b_k, a_k)

        # To avoid clutter, plot up to 30 faded harmonics
        max_to_plot = min(K, 30)
        for kk in range(1, max_to_plot+1):
            term = A_k[kk-1] * np.cos(kk*x - phi_k[kk-1])
            ax.plot(x, term, linewidth=0.8, alpha=0.35)

        if K > max_to_plot:
            ax.text(0.02, 0.04, f"… plus {K - max_to_plot} more harmonics",
                    transform=ax.transAxes)

    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.show()

    # Small readout for coefficients (first few)
    shown = min(6, K)
    A = np.sqrt(a_k[:shown]**2 + b_k[:shown]**2)
    PH = np.arctan2(b_k[:shown], a_k[:shown])
    rows = [
        "<tr><th>k</th><th>A_k</th><th>ϕ_k (rad)</th></tr>"
    ] + [
        f"<tr><td>{i}</td><td>{A[i-1]:.4g}</td><td>{PH[i-1]:.4g}</td></tr>"
        for i in range(1, shown+1)
    ]
    html = f"""
    <div style="font-family: ui-sans-serif, system-ui; line-height:1.35;">
      <b>Series summary:</b>
      a₀/2 = {a0/2:.4g},  w/π = {w_over_pi:.3f},  shift φ₀ = {phi0:.3f} rad
      <table style="margin-top:6px;border-collapse:collapse" border="1" cellpadding="4">
        {''.join(rows)}
      </table>
      <div style="margin-top:6px; font-size: 90%;">
        Note: shifting the pulse by φ₀ mixes cosine/sine terms:
        a_k = a_k⁰ cos(kφ₀), b_k = a_k⁰ sin(kφ₀), so each harmonic appears as
        A_k cos(kx − ϕ_k).
      </div>
    </div>
    """
    display(HTML(html))

interact(
    boxcar_fourier_demo,
    N=IntSlider(value=12, min=1, max=60, step=1, description="Harmonics N"),
    w_over_pi=FloatSlider(value=0.40, min=0.05, max=0.95, step=0.01, description="Half-width w/π"),
    phi0=FloatSlider(value=0.0, min=-3.1416, max=3.1416, step=0.01, description="Shift φ₀ (rad)"),
    show_harmonics=Checkbox(value=True, description="Show individual harmonics"),
)


interactive(children=(IntSlider(value=12, description='Harmonics N', max=60, min=1), FloatSlider(value=0.4, de…

<function __main__.boxcar_fourier_demo(N=12, w_over_pi=0.4, phi0=0.0, show_harmonics=True)>

# Fourier shift theorem 

**Time shift ⇒ linear phase.**  
If $y(t)=x(t-t_0)$, then
$$
\mathcal{F}\{y(t)\}(f)=Y(f)=X(f)\,e^{-i2\pi f t_0}.
$$
The **magnitude** is unchanged, $|Y(f)|=|X(f)|$; the **phase** adds a linear ramp $-2\pi f t_0$.

**Frequency shift (modulation) ⇒ spectral translation.**  
If $y(t)=x(t)\,e^{i2\pi f_0 t}$, then
$$
Y(f)=X(f-f_0),
$$
i.e., the spectrum shifts by $f_0$.

---

## Discrete versions

For a length-$N$ sequence $x[n]$ with DFT $X[k]=\sum_{n=0}^{N-1}x[n]e^{-i2\pi kn/N}$:

**(1) Circular time shift** $y[n]=x\big[(n-n_0)\!\!\mod N\big]$  
$$
Y[k]=X[k]\;e^{-i2\pi k n_0/N}.
$$
So $|Y[k]|=|X[k]|$, and $\angle Y[k]=\angle X[k]-2\pi k n_0/N$ (a linear phase ramp).

**(2) Discrete modulation** $y[n]=x[n]\,e^{i2\pi k_0 n/N}$
$$
Y[k]=X\big[(k-k_0)\!\!\mod N\big].
$$
If $k_0$ is not an integer, you still get a shift, but energy spreads across bins (spectral **leakage**).

---

## Proof

For a time shift:
$$
Y(f)=\int x(t-t_0)\,e^{-i2\pi ft}\,dt
=\int x(u)\,e^{-i2\pi f(u+t_0)}\,du
=e^{-i2\pi ft_0}\!\!\int x(u)\,e^{-i2\pi fu}\,du
=e^{-i2\pi ft_0}X(f).
$$


## What to look for in the demo

- When you **shift in time**, the **spectrum’s magnitude stays the same**, but the **phase becomes an ideal straight line** with slope $-2\pi t_0$ (continuous) or $-2\pi n_0/N$ per bin (discrete).
- When you **modulate** by $e^{i2\pi f_0 n}$, the **magnitude spectrum slides** left/right by $f_0$ cycles/sample; if $f_0 N$ is an integer, the shift is a perfect circular reindexing.


In [ ]:

%matplotlib inline
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox
from IPython.display import clear_output, display, HTML
import matplotlib.pyplot as plt

def shift_theorem_demo_fixed(n0=0, sigma=40.0, show_abs_phase=False, show_prediction=True):
    """
    Time shift: y[n] = x[(n - n0) mod N]  ⇒  Y[k] = X[k] · e^{-j 2π k n0 / N}
    This version:
      • Uses phase of the complex product Y * conj(X) to get a stable phase difference
      • Masks bins where |X| or |Y| are tiny
      • (Optional) shows absolute phases, but beware they are 0/π for real-even x[n]
    """
    clear_output(wait=True)

    N = 2048
    n = np.arange(N)
    n0 = int(np.round(n0))

    # gaussian function
    d = np.minimum(n, N - n).astype(float)
    x = np.exp(-0.5 * (d / max(1.0, sigma))**2)
    x = np.roll(x, N//2)

    # circular time shift by integer n0
    y = np.roll(x, n0)

    # FFTs and centered frequency axis (cycles/sample)
    X = np.fft.fft(x)
    Y = np.fft.fft(y)
    f = np.fft.fftfreq(N, d=1.0)
    f_c = np.fft.fftshift(f)
    Xc = np.fft.fftshift(X)
    Yc = np.fft.fftshift(Y)

    magX = np.abs(Xc)
    magY = np.abs(Yc)

    # relative phase: angle(Y * conj(X)) 
    rel = Yc * np.conj(Xc)
    rel_phase = np.angle(rel)
    rel_phase = np.unwrap(rel_phase)

    # mask bins with tiny magnitude (phase undefined there)
    mask = (magX > 1e-10) & (magY > 1e-10)
    rel_plot = np.where(mask, rel_phase, np.nan)

    # predicted linear ramp: −2 pi f n0
    ramp = -2 * np.pi * f_c * n0

    # align near DC to reduce any small constant offset
    dc_win = np.abs(f_c) < (3.0 / N)   # a few bins around DC
    if np.any(mask & dc_win):
        offset = np.nanmean(rel_plot[mask & dc_win] - ramp[mask & dc_win])
        rel_plot = rel_plot - offset

    fig, axs = plt.subplots(3, 1, figsize=(9, 9), gridspec_kw=dict(hspace=0.35))

    # Time domain
    axs[0].plot(n, x, label="x[n]")
    axs[0].plot(n, y, label=f"x[n−{n0}] (circular)", alpha=0.95)
    axs[0].set_xlim(0, N-1)
    axs[0].set_xlabel("n (samples)")
    axs[0].set_ylabel("amplitude")
    axs[0].grid(True, linestyle=":")
    axs[0].legend()

    # Magnitudes
    axs[1].plot(f_c, magX, label="|X(f)|")
    axs[1].plot(f_c, magY, label="|Y(f)|", alpha=0.95)
    axs[1].set_xlabel("frequency (cycles/sample)")
    axs[1].set_ylabel("magnitude")
    axs[1].grid(True, linestyle=":")
    axs[1].legend()

    # Phases
    if show_abs_phase:
        phX = np.angle(Xc); phY = np.angle(Yc)
        axs[2].plot(f_c, phX, label="∠X (wrapped)")
        axs[2].plot(f_c, phY, label="∠Y (wrapped)", alpha=0.95)

    axs[2].plot(f_c, rel_plot, label="∠(Y · X*)  =  ∠Y − ∠X", alpha=0.9)
    if show_prediction:
        axs[2].plot(f_c, ramp, linestyle="--", linewidth=1.2, label="predicted ramp −2π f n₀")

    axs[2].set_xlabel("frequency (cycles/sample)")
    axs[2].set_ylabel("phase (radians)")
    axs[2].set_ylim(-1000, 1000)
    axs[2].grid(True, linestyle=":")
    axs[2].legend()
    plt.show()

    display(HTML(
        "<div style='font-family:ui-sans-serif,system-ui'>"
        "Tip: absolute phase of a real-even x[n] toggles between 0 and π; "
        "the shift theorem is best seen in the relative phase ∠(Y·X*), which "
        "should align with the dashed line.</div>"
    ))

interact(
    shift_theorem_demo_fixed,
    n0=IntSlider(value=0, min=-256, max=256, step=1, description="Time shift n₀"),
    sigma=FloatSlider(value=40.0, min=5.0, max=160.0, step=1.0, description="Pulse width σ"),
    show_abs_phase=Checkbox(value=False, description="Show absolute phases"),
    show_prediction=Checkbox(value=True, description="Show prediction"),
)


interactive(children=(IntSlider(value=0, description='Time shift n₀', max=256, min=-256), FloatSlider(value=40…

<function __main__.shift_theorem_demo_fixed(n0=0, sigma=40.0, show_abs_phase=False, show_prediction=True)>

# Aliasing in sampled signals and DFTs

When you sample a continuous-time signal $x(t)$ at sampling rate $f_s$ (samples/second), you observe a discrete-time sequence $x[n]=x(n/f_s)$. Sampling causes the continuous-time spectrum to **replicate every $f_s$ Hz**. Any spectral energy above the **Nyquist limit** $f_s/2$ folds back (aliases) into $[0,\,f_s/2]$.

**Key facts**

- **Spectral periodicity:** The discrete-time Fourier transform (DTFT) of $x[n]$ is periodic with period $f_s$ in Hz (or $2\pi$ in rad/sample).
- **Aliased frequency:** A tone at frequency $f_0$ Hz appears (after sampling) at the **aliased** frequency
  $$
  f_{\text{alias}} \;=\; \left|\,\left(\,(f_0 + \tfrac{f_s}{2}) \bmod f_s \right) - \tfrac{f_s}{2}\,\right|
  \;\in\; [\,0,\,f_s/2\,).
  $$
  Equivalently, choose an integer $m$ so that $f_0 - m f_s \in [-f_s/2, f_s/2)$; that value (in magnitude) is the aliased location.
- **Nyquist–Shannon condition:** To avoid aliasing, signals must be bandlimited to $< f_s/2$. Otherwise, different continuous-time frequencies become **indistinguishable** after sampling.
- **DFT view:** For an $N$-point DFT with bin centers $f_k = k\,f_s/N$, you only see frequencies modulo $f_s$. Aliased content lands in bins near $f_{\text{alias}}$ (subject to **leakage** if the tone does not sit exactly on a bin).
- **Aliasing vs. leakage:**  
  *Aliasing* is fold-over caused by insufficient $f_s$.  
  *Leakage* is spreading caused by finite-length observation and windowing when the tone is off-bin.

**Example (intuition).**  
A 900 Hz tone sampled at $f_s=1{,}000$ Hz aliases to
$$
f_{\text{alias}} = \left|\,\big(900+500\bmod 1000\big)-500\,\right|=|400-500|=100\text{ Hz}.
$$
You will *measure* a 100 Hz tone in the DFT, even though the physical signal was 900 Hz. So, you need to increase $f_s$ to *at least* $2 \times 900$ Hz, or $1{,},800$ Hz to ensure a 900 Hz tone is measured. 


In [ ]:

%matplotlib inline
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox
from IPython.display import clear_output, display, HTML
import matplotlib.pyplot as plt

def replicas_view(
    f0=900.0,                 # tone frequency (Hz)
    fs=800.0,                 # sampling rate (Hz)
    f_max=3000.0,
    N=2048,                   # samples captured
    periods=3,                # how many DTFT periods to show (odd number recommended)
    window=True,              # Hann window (reduces leakage)
    db_scale=True             # dB magnitude (peak-normalized)
):
    """
    Show the FFT magnitude for one period [-fs/2, fs/2], then tile it across
    'periods' copies to visualize the DTFT's fs-periodic replicas.
    """
    clear_output(wait=True)

    # --- build sampled signal ---
    fs = float(fs); N = int(N); periods = int(periods)
    t  = np.arange(N)/fs
    x  = np.cos(2*np.pi*f0*t)

    # optional window
    if window:
        w = 0.5 - 0.5*np.cos(2*np.pi*np.arange(N)/(N-1))
        xw = x * w
        scale = w.sum()/N
    else:
        xw = x
        scale = 1.0

    # --- full FFT (one period, centered) ---
    Xc = np.fft.fftshift(np.fft.fft(xw))
    fc = np.fft.fftshift(np.fft.fftfreq(N, d=1/fs))   # Hz in [-fs/2, fs/2)

    mag = np.abs(Xc) / (N * max(scale, 1e-12))
    if db_scale:
        magp = 20*np.log10(mag + 1e-12)
        magp -= magp.max()  # normalize to 0 dB peak
        ylab = "magnitude (dB, peak=0)"
    else:
        magp = mag
        ylab = "magnitude (linear)"

    # --- plot multiple periods by shifting the same curve by m*fs ---
    if periods < 1:
        periods = 1
    # choose symmetric period indices if possible, e.g., periods=3 -> m in {-1,0,1}
    m0 = -(periods//2)
    ms = range(m0, m0 + periods)

    fig = plt.figure(figsize=(10, 4.6))
    ax  = plt.gca()
    for m in ms:
        ax.plot(fc + m*fs, magp)

    # helpful vertical guides at integer multiples of fs
    fmin = (m0-0.5)*fs
    fmax = (m0+periods-0.5)*fs
    for m in ms:
        ax.axvline(m*fs, linestyle="--", linewidth=1)

    # annotate alias location (fold into [-fs/2, fs/2) then copy visually)
    fa_signed = ((f0 + fs/2) % fs) - fs/2
    fa = abs(fa_signed)
    for m in ms:
        ax.axvline(m*fs + fa, linestyle=":", linewidth=1)
        ax.axvline(m*fs - fa, linestyle=":", linewidth=1)

    # ax.set_xlim(fmin, fmax)
    ax.set_xlim(-f_max, f_max)
    ax.set_xlabel("frequency (Hz)")
    ax.set_ylabel(ylab)
    ax.grid(True, linestyle=":")
    ax.set_title(f"DTFT periodicity: spectrum replicated every fs (fs = {fs:.1f} Hz)")

    plt.show()

    note = (f"You are seeing the same baseband spectrum repeated every f_s = {fs:.1f} Hz. "
            f"A {f0:.1f} Hz tone folds to ±{fa:.1f} Hz inside each period (alias).")
    display(HTML(f"<div style='font-family:ui-sans-serif,system-ui;line-height:1.35'>{note}</div>"))

# --- widget wrapper ---
interact(
    replicas_view,
    f0=FloatSlider(value=900.0, min=0.0, max=5000.0, step=1.0, description="f₀ (Hz)"),
    fs=FloatSlider(value=1000.0, min=100.0, max=3000.0, step=10.0, description="f_s (Hz)"),
    f_max=FloatSlider(value=3000.0, min=100.0, max=6000.0, step=100.0, description="maximum frequency to plot (Hz)"),
    N=IntSlider(value=2048, min=256, max=8192, step=256, description="N samples"),
    periods=IntSlider(value=3, min=1, max=9, step=1, description="# periods"),
    window=Checkbox(value=False, description="Hann window"),
    db_scale=Checkbox(value=False, description="dB scale"),
)


interactive(children=(FloatSlider(value=900.0, description='f₀ (Hz)', max=5000.0, step=1.0), FloatSlider(value…

# Aliasing in MRI

We have been discussing frequency components (units of Hz) of discrete time-varying signals (units of seconds). In MRI, our signals (images) that we want to generate have units of meters. The signal measured in MRI is in the $spatial frequency$ (k-space) domain (with units of 1/m). Note the similar reciprocal relationship between seconds and Hz (1/seconds) and the meters and 1/meters units. This is why we can use the Fourier transform to create an image!

To save scan time, it might be tempting to reduce the number of k-space samples that we acquire. But, this effectively changes the sampling frequency ($f_s$). 

The Nyquist limit tells us that the distance in $k$-space between two samples needs to be $1/FOV$ where FOV is the "field-of-view", or the distance in space between the left and right edges of our image. Below, we can see what happens when we change this step size to $n/FOV$ with $n \in [0, 1, ..., 6]$.

In [ ]:
%matplotlib inline
import h5py 
import matplotlib.pyplot as plt 
import numpy as np 
from ipywidgets import interact, IntSlider
from IPython.display import clear_output, HTML, display
import os 

# load k-space data
# fname = '/Users/nmickevicius/dev/MCW_BIOP_03-238_MRI/data/fbirn_32ch_kspace.h5'
!wget https://github.com/nmickevicius/MCW_BIOP_03-238_MRI/blob/main/data/fbirn_32ch_kspace.h5
fname = os.path.join(os.getcwd(), 'fbirn_32ch_kspace.h5')
with h5py.File(fname, 'r') as F:
    ksp = F['ksp'][:]

# function for centered inverse fourier transform
IFT = lambda x, dim: np.fft.ifftshift(np.fft.ifft(np.fft.ifftshift(x, axes=(dim,)), axis=dim), axes=(dim,))
IFT2 = lambda x: IFT(IFT(x, 0), 1)

# Convenience for stable grayscale display of k-space magnitude (log)
def _kspace_log(img):
    return np.log10(np.abs(img) + 1e-6)

# Root-sum-of-squares (RSS) over coils
def rss(imgc):
    # imgc: (Ny, Nx, Nc) complex image per coil
    return np.sqrt(np.sum(np.abs(imgc)**2, axis=-1))

# Build 2D Cartesian undersampling mask with stride Ry (rows) and Rx (cols),
# centered so the DC sample is retained.
def undersample_mask(shape, Ry=1, Rx=1):
    Ny, Nx = shape[:2]
    cy, cx = Ny // 2, Nx // 2
    yy = np.arange(Ny)
    xx = np.arange(Nx)
    mask_y = ((yy - cy) % max(1, Ry) == 0)
    mask_x = ((xx - cx) % max(1, Rx) == 0)
    return np.outer(mask_y, mask_x).astype(bool)

# Precompute full k-space coil 5 display scaling (for consistent contrast)
coil_idx = 4  # “coil 5” in 0-based indexing
full_k5_log = _kspace_log(ksp[:, :, coil_idx])
vmin_k, vmax_k = np.percentile(full_k5_log, [5, 99.9])  # robust range from full data

def show_undersampling(Ry=2, Rx=2):
    clear_output(wait=True)

    Ny, Nx, Nc = ksp.shape
    mask = undersample_mask((Ny, Nx), Ry=Ry, Rx=Rx)

    # Apply mask to all coils
    k_us = ksp * mask[:, :, None]

    # Zero-filled recon per coil, then RSS
    imgc = IFT2(k_us)  # (Ny, Nx, Nc)
    rss_img = rss(imgc)
    # Normalize RSS for display
    rss_img_disp = rss_img / (rss_img.max() + 1e-12)

    # Prepare coil 5 k-space (full and undersampled) log magnitudes
    k5_full_log = full_k5_log
    k5_us_log   = _kspace_log(k_us[:, :, coil_idx])

    # --- Plots ---
    fig, axs = plt.subplots(1, 3, figsize=(14, 4.4))
    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])

    axs[0].imshow(k5_full_log, vmin=vmin_k, vmax=vmax_k, origin='lower')
    axs[0].set_title("Full k-space (coil 5)")

    axs[1].imshow(k5_us_log, vmin=vmin_k, vmax=vmax_k, origin='lower')
    axs[1].set_title(f"Undersampled k-space (coil 5)\nRy={Ry}, Rx={Rx} (R={Ry*Rx}×)")

    axs[2].imshow(rss_img_disp, cmap='gray', origin='lower')
    axs[2].set_title("Aliased image (RSS over coils)")

    plt.tight_layout()
    plt.show()

    # Text summary
    kept = mask.sum()
    total = Ny * Nx
    accel = Ry * Rx
    display(HTML(
        f"<div style='font-family:ui-sans-serif,system-ui'>"
        f"Sampling mask keeps <b>{kept}</b> / {total} k-space points "
        f"({100*kept/total:.1f}% of lines). Acceleration R = <b>{accel}×</b>."
        f"</div>"
    ))

interact(
    show_undersampling,
    Ry=IntSlider(value=1, min=1, max=6, step=1, description="Row factor Ry"),
    Rx=IntSlider(value=1, min=1, max=6, step=1, description="Col factor Rx"),
)


interactive(children=(IntSlider(value=1, description='Row factor Ry', max=6, min=1), IntSlider(value=1, descri…

<function __main__.show_undersampling(Ry=2, Rx=2)>